# Weekend Movie Trip

a project for EECS 731 by Benjamin Wyss

Examining MovieLens movie data to build a clustering model which can recommend movies similar to other movies

###### python imports

In [1]:
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
plt.close('all')

### Reading Data Sets From CSV

MovieLens Small Datasets - Movies, Ratings, and Tags

Taken from: https://grouplens.org/datasets/movielens/ on 9/23/20

## Exploratory Data Analysis

### Cleaning the data sets


### Transforming the data sets


### Visualizing the data sets

First, calculate some basic statistics of the data set and measure correlation coefficients

### Model Construction


### Results
